In [1]:
from sentence_transformers import SentenceTransformer, SimilarityFunction
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from collections import Counter
from tqdm.auto import tqdm
import torch
import warnings
import re
import numpy as np
warnings.filterwarnings('ignore')
tqdm.pandas()

/home/dimk/my_project_dir/myenv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score, max_error

In [3]:
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
import numpy as np

In [4]:
path = ''
data = pd.read_csv('train_events.csv')
video = pd.read_csv('video_info_v2.csv')
targets = pd.read_csv('train_targets.csv')

In [5]:
title_video = video['title'].tolist()

In [6]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model = SentenceTransformer('intfloat/multilingual-e5-small', device=device)

In [7]:
# embeddings = model.encode(title_video)

In [8]:
# embeddings.shape

In [9]:
# video['embedings_v1'] =  list(np.array(embeddings))

In [10]:
# video.to_parquet('video_info_v2_emb.parquet', index=False)

In [11]:
video = pd.read_parquet('video_info_v2_emb.parquet')

In [12]:
# объеденяем данные
df_video_sex = data[['rutube_video_id','viewer_uid']].merge(video[['rutube_video_id','embedings_v1']],on='rutube_video_id').merge(targets[['viewer_uid','sex']],on='viewer_uid')
df_video_sex['sex'] = df_video_sex['sex'].replace({'female':-1, 'male':1})
df_video_sex = df_video_sex.groupby(['rutube_video_id'])['sex'].agg(['sum','count']).reset_index()
df_video_sex = df_video_sex.merge(video[['rutube_video_id','embedings_v1']], on='rutube_video_id')

n=10 #минимальное кол-во просмотров у видео


#Выделяем датасет из видео по кол-вам просмотров
df_video_sex_1 = df_video_sex[df_video_sex['count'] >= n][((df_video_sex['count'] - df_video_sex['sum'].abs())/df_video_sex['count']) > 0.8]
df_video_sex_1['target_sex'] = 3
df_video_sex_2 = df_video_sex[df_video_sex['count'] >= n][((df_video_sex['count'] - df_video_sex['sum'].abs())/df_video_sex['count']) <= 0.8]
df_video_sex_2['target_sex'] = df_video_sex_2['sum'] > 0
df_video_sex_text = pd.concat([df_video_sex_2,df_video_sex_1])

# 0- видео которые смотрят чаще женщины
# 1- видео которые смотрят чаще мужчины
# 3- видео которые смотрят все


# Разделение данных на обучающую и тестовую выборки
X = df_video_sex_text['embedings_v1'].values.tolist()  # Тексты (названия)

y = df_video_sex_text['target_sex']  # Метки классов

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# Инициализация и обучение модели логистической регрессии
# model = LogisticRegression(max_iter=1000)
model = CatBoostClassifier(verbose=100, iterations=3000)
model.fit(X_train, y_train)

# Предсказания на тестовой выборке
y_pred = model.predict(X_test)

# Оценка модели
accuracy = accuracy_score(y_test, y_pred)

print(f'Размер датасета: {df_video_sex_text.shape}')
print(f'Accuracy: {accuracy:.4f}')
print(classification_report(y_test, y_pred))
model_sex = CatBoostClassifier(verbose=100, iterations=3000)
model_sex.fit(X, y)


Learning rate set to 0.035925
0:	learn: 1.0870794	total: 87.6ms	remaining: 4m 22s
100:	learn: 0.7509470	total: 4.1s	remaining: 1m 57s
200:	learn: 0.6564360	total: 8.37s	remaining: 1m 56s
300:	learn: 0.5930514	total: 13s	remaining: 1m 56s
400:	learn: 0.5418428	total: 17.7s	remaining: 1m 54s
500:	learn: 0.5048145	total: 22.4s	remaining: 1m 51s
600:	learn: 0.4727641	total: 27s	remaining: 1m 47s
700:	learn: 0.4467000	total: 31.7s	remaining: 1m 43s
800:	learn: 0.4235990	total: 36.5s	remaining: 1m 40s
900:	learn: 0.4046065	total: 41.1s	remaining: 1m 35s
1000:	learn: 0.3873258	total: 45.8s	remaining: 1m 31s
1100:	learn: 0.3712844	total: 50.4s	remaining: 1m 26s
1200:	learn: 0.3566951	total: 54.9s	remaining: 1m 22s
1300:	learn: 0.3433225	total: 59.5s	remaining: 1m 17s
1400:	learn: 0.3302956	total: 1m 4s	remaining: 1m 13s
1500:	learn: 0.3183042	total: 1m 8s	remaining: 1m 8s
1600:	learn: 0.3070970	total: 1m 13s	remaining: 1m 4s
1700:	learn: 0.2969384	total: 1m 17s	remaining: 59.5s
1800:	learn: 0.

In [13]:
# объеденяем данные
df_video_age = data[['rutube_video_id','viewer_uid']].merge(video[['rutube_video_id','title']],on='rutube_video_id').merge(targets[['viewer_uid','age']],on='viewer_uid')
df_video_age = df_video_age.groupby(['rutube_video_id', 'title'])['age'].agg(['median','count']).reset_index()
df_video_age = df_video_age.merge(video[['rutube_video_id','embedings_v1']], on='rutube_video_id')

n=10 #минимальное кол-во просмотров у видео

# Разделение данных на обучающую и тестовую выборки
X = df_video_age[df_video_age['count'] >= n]['embedings_v1'].values.tolist()  # Тексты (названия)
y = df_video_age[df_video_age['count'] >= n]['median']  # Метки классов

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Инициализация и обучение модели логистической регрессии
model = CatBoostRegressor(verbose=100, iterations=3000)
model.fit(X_train, y_train)

# Предсказания на тестовой выборке
y_pred = model.predict(X_test)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
max_err = max_error(y_test, y_pred)
print()
# print(f'Размер датасета: {X.shape}')
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')
print(f'Maximum Error: {max_err}')
print()
model_age = CatBoostRegressor(verbose=100, iterations=3000)
model_age.fit(X, y)

Learning rate set to 0.025757
0:	learn: 5.1795767	total: 12.9ms	remaining: 38.5s
100:	learn: 4.2700832	total: 1.35s	remaining: 38.7s
200:	learn: 3.9777152	total: 2.57s	remaining: 35.9s
300:	learn: 3.7844512	total: 3.79s	remaining: 34s
400:	learn: 3.6263318	total: 5.04s	remaining: 32.7s
500:	learn: 3.4633306	total: 6.32s	remaining: 31.5s
600:	learn: 3.3246170	total: 7.7s	remaining: 30.7s
700:	learn: 3.1968376	total: 8.97s	remaining: 29.4s
800:	learn: 3.0858402	total: 10.2s	remaining: 28s
900:	learn: 2.9813650	total: 11.4s	remaining: 26.6s
1000:	learn: 2.8821516	total: 12.9s	remaining: 25.7s
1100:	learn: 2.7919140	total: 14.4s	remaining: 24.8s
1200:	learn: 2.7069390	total: 15.9s	remaining: 23.8s
1300:	learn: 2.6269117	total: 17.4s	remaining: 22.7s
1400:	learn: 2.5503342	total: 18.9s	remaining: 21.6s
1500:	learn: 2.4796824	total: 20.7s	remaining: 20.6s
1600:	learn: 2.4112827	total: 22.5s	remaining: 19.7s
1700:	learn: 2.3444079	total: 24.3s	remaining: 18.6s
1800:	learn: 2.2851178	total: 26

In [14]:
# объеденяем данные
df_video_age_class = data[['rutube_video_id','viewer_uid']].merge(video[['rutube_video_id','title']],on='rutube_video_id').merge(targets[['viewer_uid','age_class']],on='viewer_uid')
df_video_age_class = df_video_age_class.groupby(['rutube_video_id', 'title'])['age_class'].agg(['median','count']).reset_index()
df_video_age_class = df_video_age_class.merge(video[['rutube_video_id','embedings_v1']], on='rutube_video_id')


n=10 #минимальное кол-во просмотров у видео

# Разделение данных на обучающую и тестовую выборки
X = df_video_age_class[df_video_age_class['count'] >= n]['embedings_v1'].values.tolist()  # Тексты (названия)

df_video_age_class.loc[df_video_age_class['count'] >= n, 'median'] = np.floor(df_video_age_class.loc[df_video_age_class['count'] >= n, 'median'])
y = df_video_age_class[df_video_age_class['count'] >= n]['median']  # Метки классов

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Инициализация и обучение модели логистической регрессии
model = CatBoostClassifier(verbose=100, iterations=3000)
model.fit(X_train, y_train)

# Предсказания на тестовой выборке
y_pred = model.predict(X_test)

# Оценка модели
accuracy = accuracy_score(y_test, y_pred)
print()
# print(f'Размер датасета: {df_video_sex_text.shape}')
print(f'Accuracy: {accuracy:.4f}')
print(classification_report(y_test, y_pred))
print()
model_age_class = CatBoostClassifier(verbose=100, iterations=3000)
model_age_class.fit(X, y)

Learning rate set to 0.035925
0:	learn: 1.3541767	total: 50.2ms	remaining: 2m 30s
100:	learn: 0.7339924	total: 5.84s	remaining: 2m 47s
200:	learn: 0.6579762	total: 11.3s	remaining: 2m 37s
300:	learn: 0.6099404	total: 16.9s	remaining: 2m 31s
400:	learn: 0.5673770	total: 22.2s	remaining: 2m 24s
500:	learn: 0.5332061	total: 27.5s	remaining: 2m 16s
600:	learn: 0.5040111	total: 32.6s	remaining: 2m 10s
700:	learn: 0.4779515	total: 37.8s	remaining: 2m 4s
800:	learn: 0.4549180	total: 43s	remaining: 1m 58s
900:	learn: 0.4342786	total: 48.3s	remaining: 1m 52s
1000:	learn: 0.4152166	total: 53.6s	remaining: 1m 47s
1100:	learn: 0.3975189	total: 59s	remaining: 1m 41s
1200:	learn: 0.3819367	total: 1m 4s	remaining: 1m 36s
1300:	learn: 0.3668075	total: 1m 10s	remaining: 1m 31s
1400:	learn: 0.3535471	total: 1m 16s	remaining: 1m 26s
1500:	learn: 0.3412960	total: 1m 22s	remaining: 1m 22s
1600:	learn: 0.3291251	total: 1m 28s	remaining: 1m 17s
1700:	learn: 0.3175961	total: 1m 34s	remaining: 1m 11s
1800:	lea

In [15]:
X_emb = pd.DataFrame(video['embedings_v1'].values.tolist()) 

video['age_class_vidio_emb'] = model_age_class.predict(X_emb)
video['age_vidio_emb'] = model_age.predict(X_emb)
video['sex_vidio_emb'] = model_sex.predict(X_emb)
video

,rutube_video_id,title,category,duration,author_id,embedings_v1,age_class_vidio_emb,age_vidio_emb,sex_vidio_emb
0,video_185549,Как собрать букет из мыльных тюльпанов - Силик...,Хобби,1559160,1015054,"[0.016559377, -0.048766337, -0.07356531, -0.04...",1.0,33.254762,0
1,video_111035,"Осторожно, Киберземляне!, 1 сезон, 12 серия",Сериалы,1320007,1002180,"[0.03826977, 0.006300223, -0.060954306, -0.040...",2.0,31.540075,0
2,video_476517,ПОПУЛЯРНЫЕ ВИДЕОИГРЫ в LEGO... перевод - TD BR...,Хобби,606145,1095337,"[0.0027630243, -0.0022597108, -0.07325064, -0....",2.0,33.746011,1
3,video_157198,"Хороший лжец (фильм, 2019)",Фильмы,6577440,1043618,"[-0.01342462, -0.034203686, -0.06691585, -0.06...",2.0,34.777196,3
4,video_289824,Нашего старого гнобят по-всякому,Развлечения,859493,1009535,"[0.031714797, -0.001258276, 0.0074235974, -0.0...",3.0,38.557864,1
...,...,...,...,...,...,...,...,...,...
481475,video_196126,14. Landscape Layer Blend,Дизайн,462000,1121338,"[0.08850326, 0.005539659, -0.054138623, -0.080...",2.0,34.512828,3
481476,video_390909,Рихард Штраус: Завтра! — Людмила Рубинская,Музыка,227628,1043510,"[0.04961556, -0.019708708, -0.07893584, -0.043...",2.0,37.918706,1
481477,video_326364,"Поймай Тинипин! Королевство эмоций, 2 серия. С...",Мультфильмы,744576,1002799,"[0.025071798, 0.024965443, -0.048421882, 0.003...",2.0,30.903862,0
481478,video_374957,ВОРКУТА. 1 СЕРИЯ. УЖАСЫ. ПСИХОЛОГИЧЕСКИЙ ТРИЛЛЕР,Люди и блоги,2597385,1008386,"[0.052029993, -0.007863531, -0.1148146, -0.020...",2.0,34.875372,0


In [16]:
video.to_parquet('video_info_v2_emb_E5.parquet', index=False)